# Cuaderno RETO 1: Exploración de los datos

En este cuaderno se realiza la **exploración inicial del dataset licencias** proporcionado
para la práctica de Bases de Datos NoSQL.

El objetivo de este reto es **comprender la estructura, el contenido y la calidad de los datos**
sin realizar todavía tareas de limpieza ni inserción en la base de datos.

Esta exploración permitirá identificar:
- Número de registros y atributos
- Tipos de datos
- Valores nulos
- Posibles claves y relaciones
- Problemas de calidad de los datos


### Contexto del conjunto de datos (Licencias)

El conjunto de datos describe los **expedientes de licencias y declaraciones responsables**
asociados a los locales registrados en el Censo de Locales y Actividades del Ayuntamiento
de Madrid.

Cada registro corresponde a un expediente administrativo identificado por un número de
referencia de licencia (`ref_licencia`) y vinculado a un local mediante su identificador
(`id_local`). Un mismo local puede tener asociados uno o varios expedientes de licencia,
reflejando su historial administrativo.

El dataset incluye información relativa al tipo de licencia o procedimiento administrativo
(licencia urbanística, licencia de funcionamiento, declaración responsable, transmisión,
etc.), así como el estado de tramitación del expediente (en tramitación, concedida,
denegada, entre otros) y la fecha del decreto o de eficacia correspondiente.

Asimismo, el fichero incorpora información de contexto del local asociada a cada
expediente, como su localización territorial, dirección y tipo de acceso, con el fin de
facilitar el análisis conjunto con otros datasets del sistema.

Los datos reflejan todos los expedientes de licencia asociados a cada local en el momento
de la extracción, independientemente de su estado de tramitación.


In [ ]:
# Imports necesarios
import pandas as pd 

# Configuración para mostrar todas las filas y columnas sin truncar
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)

### Carga del dataset

En primer lugar se carga el fichero CSV.
Este dataset contiene datos relativos a las terrazas de hostelería y restauración 
que constan asociadas a locales del censo de locales y actividades del Ayuntamiento de 
Madrid.

El fichero utiliza el carácter `;` como separador de campos.

In [ ]:
df_licencias = pd.read_csv("../../data/raw/licencias202312.csv", sep=';')

# Mostramos el tamaño del DataFrame (número de filas y columnas)
print(df_licencias.shape)

El resultado anterior indica que en total tenemos 150829 filas y 49 columnas presentes en el dataset.

A continuación vamos a visualizar las primeras filas del dataset.

In [ ]:
# Permite obtener una primera idea del contenido y formato de los datos
df_licencias.head()

A continuación se muestra una muestra aleatoria de registros.
Esto permite observar valores representativos que no necesariamente
aparecen en las primeras filas del fichero.

In [ ]:
df_licencias.sample(5, random_state=42)

### Estructura y tipos de datos

Se analiza la estructura del DataFrame para identificar:
- Nombre de los atributos
- Tipo de dato asignado por pandas
- Número de valores no nulos por columna

In [ ]:
df_licencias.info()

A partir de esta información se puede observar que:
- Existen 4 columnas de tipo float. 
- Existen 21 columnas de tipo int64. 
- Existen 24 columnas de tipo str. 
- Los campos que representan fechas están representados como str.

### Análisis de valores nulos

Se comprueba la existencia de valores nulos en el dataset.
En esta fase **no se eliminan ni corrigen**, únicamente se identifican.

In [ ]:
df_licencias.isnull().sum()[df_licencias.isnull().sum() > 0]

El resultado muestra qué columnas contienen valores nulos y cuántos registros se ven afectados en cada caso. Para este caso podemos ver que hay algunos identificadores (seguro se utilizarán dentro de alguna "relación") y coordenadas sin valores.

### Análisis de columnas de texto

Se identifican las columnas de tipo texto para analizar posibles problemas de calidad, como espacios en blanco innecesarios o formatos inconsistentes.

In [ ]:
text_cols = df_licencias.select_dtypes(include="object").columns
print(text_cols)

A continuación se inspeccionan los valores únicos de una columna de texto representativa para observar posibles inconsistencias.

In [ ]:
df_licencias["desc_barrio_local"].unique()

En los valores mostrados se puede observar la presencia de espacios en blanco
al final de las cadenas de texto, lo que indica que será necesario realizar
un preprocesado mínimo antes de la inserción en la base de datos.

### Identificación de columnas de fecha

Se identifican las columnas que contienen información temporal.
Estas columnas suelen comenzar por el prefijo `fx_` o `fecha`.

In [ ]:
date_cols = [c for c in df_licencias.columns if "fx_" in c.lower() or "fecha" in c.lower()]
print(date_cols)

Las columnas identificadas contienen fechas almacenadas como texto,
por lo que será necesario convertirlas a un formato de fecha adecuado
en fases posteriores para permitir consultas temporales.

### Listado completo de columnas

A continuación, se muestra el listado completo de atributos del dataset para identificar posibles claves y relaciones con otros ficheros.

In [ ]:
# Crea una tabla con dos columnas: "Columna" y "Tipo"
tipos_df = df_licencias.dtypes.reset_index()
tipos_df.columns = ['Columna', 'Tipo de Dato']

tipos_df.style.set_properties(**{'text-align': 'left'}).hide(axis='index')

### Conclusión del dataset y campos relevantes para el modelo de datos

A partir de la exploración realizada, se identifican aquellos campos que
resultan especialmente relevantes para la construcción del modelo de datos,
ya sea por su papel como identificadores, por permitir relaciones con otros
datasets o por aportar información estructural clave.


In [ ]:
cols_licencias_clave = [
    # Identificación y relación principal
    "id_local",

    # Contexto territorial
    "id_distrito_local",
    "desc_distrito_local",
    "id_barrio_local",
    "desc_barrio_local",
    "id_seccion_censal_local",

    # Clasificación administrativa del local
    "id_tipo_acceso_local",
    "desc_tipo_acceso_local",
    "id_situacion_local",
    "desc_situacion_local",

    # Dirección del edificio
    "id_ndp_edificio",
    "id_vial_edificio",
    "clase_vial_edificio",
    "desc_vial_edificio",
    "num_edificio",
    "cal_edificio",
    "secuencial_local_PC",

    # Dirección de acceso al local
    "id_ndp_acceso",
    "id_vial_acceso",
    "clase_vial_acceso",
    "desc_vial_acceso",
    "num_acceso",
    "cal_acceso",

    # Agrupaciones
    "id_agrupacion",
    "nombre_agrupacion",
    "id_tipo_agrup",
    "desc_tipo_agrup",
    "id_planta_agrupado",
    "id_local_agrupado",

    # Identificación comercial
    "rotulo",

    # Información específica de la licencia
    "ref_licencia",
    "id_tipo_licencia",
    "desc_tipo_licencia",
    "id_tipo_situacion_licencia",
    "desc_tipo_situacion_licencia",
    "Fecha_Dec_Lic",

    # Información temporal del registro
    "fx_datos_ini",
    "fx_datos_fin"
]

df_licencias[cols_licencias_clave].drop_duplicates().sample(10, random_state=42)

Se han seleccionado aquellos campos que permiten
relacionar de forma clara la información de las licencias con los locales a los
que están asociadas, siendo `id_local` el identificador clave que actúa como
nexo con el resto de datasets del proyecto.

Se incluyen los atributos territoriales y administrativos del local con el
objetivo de mantener la coherencia con los datasets de locales, terrazas y
actividad económica, facilitando así la integración de la información en un
modelo de datos común.

La información de dirección, tanto del edificio como del acceso real al local,
se conserva debido a que puede variar entre locales y resultar relevante en
casos de agrupaciones o accesos diferenciados.

Los campos específicos de la licencia (`ref_licencia`, tipo de licencia y
estado de tramitación) constituyen el núcleo informativo de este dataset, ya que
permiten identificar el expediente administrativo, su naturaleza y su situación
actual.

Asimismo, se incluye la fecha de decreto de la licencia, que resulta relevante
para análisis temporales y para entender la evolución administrativa de los
locales.

Esta selección no pretende ser definitiva para el modelo final, sino que sirve
como base para analizar relaciones, cardinalidades y posibles estructuras de
embebido o referencia en fases posteriores del diseño del modelo de datos NoSQL.
